# Preprocessing of the data

This report follows the structure of the CRISP-DM. The details of each and the in-depth explanation is presented in the report. In this notebook, comments are made for the purpose of possible reconstruction and to follow the thoughts of the code.

Important to note, that given the large dataset, a visualisation tool was used for the very first part of the data understanding of the data set.

### Importing of the basics

In [1]:
# Importing the basics
import pandas as pd
import numpy as np


### Presentation of the data

In [2]:
# Reading the data file, from the same folder as the jupyter Notebook.
SBA_df = pd.read_csv('SBAnational original.csv')

#Displaying the data, standard is displaying the first 5 rows
SBA_df.head()

C:\Users\lynma\AppData\Local\Temp\ipykernel_4048\521627954.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  SBA_df = pd.read_csv('SBAnational original.csv')


LoanNr_ChkDgt                           Name          City State    Zip  \
0     1000014003                 ABC HOBBYCRAFT    EVANSVILLE    IN  47711   
1     1000024006    LANDMARK BAR & GRILLE (THE)     NEW PARIS    IN  46526   
2     1000034009          WHITLOCK DDS, TODD M.   BLOOMINGTON    IN  47401   
3     1000044001  BIG BUCKS PAWN & JEWELRY, LLC  BROKEN ARROW    OK  74012   
4     1000054004    ANASTASIA CONFECTIONS, INC.       ORLANDO    FL  32801   

                            Bank BankState   NAICS ApprovalDate ApprovalFY  \
0               FIFTH THIRD BANK        OH  451120    28-Feb-97       1997   
1                1ST SOURCE BANK        IN  722410    28-Feb-97       1997   
2        GRANT COUNTY STATE BANK        IN  621210    28-Feb-97       1997   
3  1ST NATL BK & TR CO OF BROKEN        OK       0    28-Feb-97       1997   
4        FLORIDA BUS. DEVEL CORP        FL       0    28-Feb-97       1997   

   ...  RevLineCr  LowDoc  ChgOffDate  DisbursementDate  DisbursementGross  \
0  ...          N       Y         NaN         28-Feb-99        $60,000.00    
1  ...          N       Y         NaN         31-May-97        $40,000.00    
2  ...          N       N         NaN         31-Dec-97       $287,000.00    
3  ...          N       Y         NaN         30-Jun-97        $35,000.00    
4  ...          N       N         NaN         14-May-97       $229,000.00    

   BalanceGross  MIS_Status ChgOffPrinGr        GrAppv      SBA_Appv  
0        $0.00        P I F       $0.00    $60,000.00    $48,000.00   
1        $0.00        P I F       $0.00    $40,000.00    $32,000.00   
2        $0.00        P I F       $0.00   $287,000.00   $215,250.00   
3        $0.00        P I F       $0.00    $35,000.00    $28,000.00   
4        $0.00        P I F       $0.00   $229,000.00   $229,000.00   

[5 rows x 27 columns]

In [3]:
# We want to see the Data types of the attributes, as they are classified by Panda
# We can identify mulitple problems with the columns/attributes. Therefore we investigate which types of columns we have. https://pbpython.com/pandas_dtypes.html
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899163 entries, 0 to 899162
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899163 non-null  int64  
 1   Name               899149 non-null  object 
 2   City               899133 non-null  object 
 3   State              899149 non-null  object 
 4   Zip                899163 non-null  int64  
 5   Bank               897604 non-null  object 
 6   BankState          897597 non-null  object 
 7   NAICS              899163 non-null  int64  
 8   ApprovalDate       899163 non-null  object 
 9   ApprovalFY         899163 non-null  object 
 10  Term               899163 non-null  int64  
 11  NoEmp              899163 non-null  int64  
 12  NewExist           899027 non-null  float64
 13  CreateJob          899163 non-null  int64  
 14  RetainedJob        899163 non-null  int64  
 15  FranchiseCode      899163 non-null  int64  
 16  Ur

In [4]:
# From the above, we learn how the data is being reading, and from here the first sorting and data preparation can begin.

### Dropping attributes

In [5]:
# Based on the first data understanding we can see that there are more than 900K instances. 
# Furthermore, some of the coloumns are non-relevant for the models. First we drop non-relevant columns.

#Updating the DF variable to not include the dropped columns
SBA_df = SBA_df.drop(columns=['City','Zip', 'CreateJob', 'RetainedJob', 'ChgOffDate', 
                     'LoanNr_ChkDgt', 'Name', 'ApprovalDate','DisbursementDate', 'BalanceGross',
                     'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'Bank','RevLineCr'])

# Seeing the result by using .info()
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899163 entries, 0 to 899162
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              899149 non-null  object 
 1   BankState          897597 non-null  object 
 2   NAICS              899163 non-null  int64  
 3   ApprovalFY         899163 non-null  object 
 4   Term               899163 non-null  int64  
 5   NoEmp              899163 non-null  int64  
 6   NewExist           899027 non-null  float64
 7   FranchiseCode      899163 non-null  int64  
 8   UrbanRural         899163 non-null  int64  
 9   LowDoc             896581 non-null  object 
 10  DisbursementGross  899163 non-null  object 
 11  MIS_Status         897166 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 82.3+ MB


### Reducing size of dataset based on the years

In [6]:
# Then we need to convert the column from objects into integers using the astype() function. https://www.linkedin.com/pulse/change-data-type-columns-pandas-mohit-sharma/
#Through an iterating process 

SBA_df["ApprovalFY"] = pd.to_numeric(SBA_df["ApprovalFY"], errors='coerce')

#The errors='coerce' creates NaN values when meeting an error. We want to drop those
SBA_df["ApprovalFY"].dropna(inplace=True)

SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899163 entries, 0 to 899162
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              899149 non-null  object 
 1   BankState          897597 non-null  object 
 2   NAICS              899163 non-null  int64  
 3   ApprovalFY         899146 non-null  float64
 4   Term               899163 non-null  int64  
 5   NoEmp              899163 non-null  int64  
 6   NewExist           899027 non-null  float64
 7   FranchiseCode      899163 non-null  int64  
 8   UrbanRural         899163 non-null  int64  
 9   LowDoc             896581 non-null  object 
 10  DisbursementGross  899163 non-null  object 
 11  MIS_Status         897166 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 82.3+ MB


In [7]:
#Converting the column into integers. https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas
SBA_df['ApprovalFY'] = SBA_df['ApprovalFY'].fillna(0).astype(np.int64)
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
0    IN        OH  451120        1997    84      4       2.0              1   
1    IN        IN  722410        1997    60      2       2.0              1   
2    IN        IN  621210        1997   180      7       1.0              1   
3    OK        OK       0        1997    60      2       1.0              1   
4    FL        FL       0        1997   240     14       1.0              1   

   UrbanRural LowDoc DisbursementGross MIS_Status  
0           0      Y       $60,000.00       P I F  
1           0      Y       $40,000.00       P I F  
2           0      N      $287,000.00       P I F  
3           0      Y       $35,000.00       P I F  
4           0      N      $229,000.00       P I F

In [8]:
# Checking if the column is the right type of Dtype
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899163 entries, 0 to 899162
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              899149 non-null  object 
 1   BankState          897597 non-null  object 
 2   NAICS              899163 non-null  int64  
 3   ApprovalFY         899163 non-null  int64  
 4   Term               899163 non-null  int64  
 5   NoEmp              899163 non-null  int64  
 6   NewExist           899027 non-null  float64
 7   FranchiseCode      899163 non-null  int64  
 8   UrbanRural         899163 non-null  int64  
 9   LowDoc             896581 non-null  object 
 10  DisbursementGross  899163 non-null  object 
 11  MIS_Status         897166 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 82.3+ MB


In [9]:
# Next we want to limit the dataset to only include instances relevant for the model. We want as recent data as possible
# but also keeping in mind, that the loans should have a fair chance of defaulting. Based on visual analysis, the range chosen is
# the years from 2003-2010. Using the drop() function. https://sparkbyexamples.com/pandas/pandas-delete-rows-based-on-column-value/

SBA_df = SBA_df.drop(SBA_df[(SBA_df['ApprovalFY'] < 2003) | (SBA_df['ApprovalFY'] > 2010)].index)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427438 entries, 12 to 896096
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              427438 non-null  object 
 1   BankState          427426 non-null  object 
 2   NAICS              427438 non-null  int64  
 3   ApprovalFY         427438 non-null  int64  
 4   Term               427438 non-null  int64  
 5   NoEmp              427438 non-null  int64  
 6   NewExist           427344 non-null  float64
 7   FranchiseCode      427438 non-null  int64  
 8   UrbanRural         427438 non-null  int64  
 9   LowDoc             425412 non-null  object 
 10  DisbursementGross  427438 non-null  object 
 11  MIS_Status         425947 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 42.4+ MB


In [10]:
#Checking if the data looks fine after preprocessing
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc DisbursementGross MIS_Status  
12           1      N      $253,400.00       P I F  
24           1      N      $137,300.00       P I F  
28           1      N      $438,541.00       P I F  
30           1      N       $51,440.00       P I F  
32           1      N       $50,000.00       P I F

### Removing all NaN and Null rows from the dataset

In [11]:
#Removing all rows with either NaN or null values https://sparkbyexamples.com/pandas/pandas-drop-rows-with-nan-values-in-dataframe/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
SBA_df.dropna()
SBA_df.dropna(inplace=True)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423820 entries, 12 to 896096
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              423820 non-null  object 
 1   BankState          423820 non-null  object 
 2   NAICS              423820 non-null  int64  
 3   ApprovalFY         423820 non-null  int64  
 4   Term               423820 non-null  int64  
 5   NoEmp              423820 non-null  int64  
 6   NewExist           423820 non-null  float64
 7   FranchiseCode      423820 non-null  int64  
 8   UrbanRural         423820 non-null  int64  
 9   LowDoc             423820 non-null  object 
 10  DisbursementGross  423820 non-null  object 
 11  MIS_Status         423820 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 42.0+ MB


After making the dataset smaller, each variable can now be configured to match the upcoming predictive models. 
### Starting with DisbursementGross

In [12]:
# Removing $ from disburementGross and then converting. https://pbpython.com/currency-cleanup.html
#First the object has to be converted into a string to remove the dollar sign. https://stackoverflow.com/questions/22231592/pandas-change-data-type-of-series-to-string
SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].astype("string")

SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423820 entries, 12 to 896096
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              423820 non-null  object 
 1   BankState          423820 non-null  object 
 2   NAICS              423820 non-null  int64  
 3   ApprovalFY         423820 non-null  int64  
 4   Term               423820 non-null  int64  
 5   NoEmp              423820 non-null  int64  
 6   NewExist           423820 non-null  float64
 7   FranchiseCode      423820 non-null  int64  
 8   UrbanRural         423820 non-null  int64  
 9   LowDoc             423820 non-null  object 
 10  DisbursementGross  423820 non-null  string 
 11  MIS_Status         423820 non-null  object 
dtypes: float64(1), int64(6), object(4), string(1)
memory usage: 42.0+ MB


In [13]:
# The column is now a string and the $ can now be removed https://stackoverflow.com/questions/38516481/trying-to-remove-commas-and-dollars-signs-with-pandas-in-python

SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].str.replace('$','', regex=True)
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc DisbursementGross MIS_Status  
12           1      N       253,400.00       P I F  
24           1      N       137,300.00       P I F  
28           1      N       438,541.00       P I F  
30           1      N        51,440.00       P I F  
32           1      N        50,000.00       P I F

In [14]:
#Removing space with the same function
SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].str.replace(' ','', regex=True)
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc DisbursementGross MIS_Status  
12           1      N        253,400.00      P I F  
24           1      N        137,300.00      P I F  
28           1      N        438,541.00      P I F  
30           1      N         51,440.00      P I F  
32           1      N         50,000.00      P I F

In [15]:
# Using the function again
SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].str.replace(',', '', regex=True)
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc DisbursementGross MIS_Status  
12           1      N         253400.00      P I F  
24           1      N         137300.00      P I F  
28           1      N         438541.00      P I F  
30           1      N          51440.00      P I F  
32           1      N          50000.00      P I F

In [16]:
#Stripping the .00 on all values by removing the last three characters. https://stackoverflow.com/questions/60229544/remove-last-char-in-a-string-for-each-row-in-pandas-df
SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].str[:-3]
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc DisbursementGross MIS_Status  
12           1      N            253400      P I F  
24           1      N            137300      P I F  
28           1      N            438541      P I F  
30           1      N             51440      P I F  
32           1      N             50000      P I F

In [17]:
#Converting the column into an integer type with the astype() function
SBA_df['DisbursementGross'] = SBA_df['DisbursementGross'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423820 entries, 12 to 896096
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   State              423820 non-null  object 
 1   BankState          423820 non-null  object 
 2   NAICS              423820 non-null  int64  
 3   ApprovalFY         423820 non-null  int64  
 4   Term               423820 non-null  int64  
 5   NoEmp              423820 non-null  int64  
 6   NewExist           423820 non-null  float64
 7   FranchiseCode      423820 non-null  int64  
 8   UrbanRural         423820 non-null  int64  
 9   LowDoc             423820 non-null  object 
 10  DisbursementGross  423820 non-null  int64  
 11  MIS_Status         423820 non-null  object 
dtypes: float64(1), int64(7), object(4)
memory usage: 42.0+ MB


In [18]:
#Testing if the data looks fine
SBA_df.head()

State BankState   NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12    NC        MN  445299        2006   162      2       2.0          15100   
24    AR        MS  722211        2006   126      7       1.0              1   
28    CA        SD  611110        2006    83     18       2.0              1   
30    FL        FL  238140        2006    84      4       1.0              1   
32    LA        LA  532490        2006    60      3       1.0              1   

    UrbanRural LowDoc  DisbursementGross MIS_Status  
12           1      N             253400      P I F  
24           1      N             137300      P I F  
28           1      N             438541      P I F  
30           1      N              51440      P I F  
32           1      N              50000      P I F

### NewExist

In [19]:
#First we count the different number of occurences of each value type
from collections import Counter
counts = Counter(SBA_df['NewExist'])
print(counts)

Counter({1.0: 301987, 2.0: 121512, 0.0: 321})


In [20]:
# NewExist has three types of values. 0, 1 and 2. 0 is equal null. 1 and 2 are true values. https://stackoverflow.com/questions/49841989/python-drop-value-0-row-in-specific-columns
SBA_df = SBA_df.loc[SBA_df['NewExist'] != 0]

print(Counter(SBA_df['NewExist']))
#We can see how there is no more 0 values.

Counter({1.0: 301987, 2.0: 121512})


In [21]:
# Using the astype() function to change the data type. https://www.geeksforgeeks.org/python-pandas-dataframe-astype/
SBA_df['NewExist'] = SBA_df['NewExist'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423499 entries, 12 to 896096
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   State              423499 non-null  object
 1   BankState          423499 non-null  object
 2   NAICS              423499 non-null  int64 
 3   ApprovalFY         423499 non-null  int64 
 4   Term               423499 non-null  int64 
 5   NoEmp              423499 non-null  int64 
 6   NewExist           423499 non-null  int64 
 7   FranchiseCode      423499 non-null  int64 
 8   UrbanRural         423499 non-null  int64 
 9   LowDoc             423499 non-null  object
 10  DisbursementGross  423499 non-null  int64 
 11  MIS_Status         423499 non-null  object
dtypes: int64(8), object(4)
memory usage: 42.0+ MB


### Franchise code

In [22]:
#Starting with counting how many instances of 0 and 1 there are. We want to add these two together, since they represent the same.
counts = Counter(SBA_df['FranchiseCode'])
print(counts)

Counter({1: 241136, 0: 161068, 78760: 1592, 68020: 1380, 17998: 501, 25650: 295, 50564: 266, 21420: 244, 81612: 228, 21780: 218, 79140: 202, 77174: 198, 84535: 196, 65140: 168, 52000: 164, 16920: 163, 22470: 162, 24850: 153, 9050: 136, 19755: 133, 38605: 131, 73675: 120, 67750: 120, 89769: 110, 10734: 108, 1350: 108, 1560: 106, 61901: 104, 68250: 104, 35706: 103, 53650: 100, 75710: 98, 52875: 96, 4516: 94, 43579: 94, 10465: 93, 50954: 93, 39598: 92, 26871: 92, 20550: 89, 51720: 89, 79902: 89, 48000: 87, 55120: 83, 21425: 81, 70130: 81, 48400: 80, 17205: 80, 68021: 79, 83882: 74, 34650: 72, 67735: 72, 49952: 68, 31415: 68, 26650: 65, 66910: 64, 10528: 63, 66320: 62, 48450: 61, 73525: 60, 59450: 60, 80025: 58, 16310: 57, 48260: 57, 36680: 56, 75988: 55, 70554: 54, 3512: 54, 61897: 52, 73000: 52, 9450: 52, 29600: 51, 25360: 51, 80979: 51, 15100: 50, 34845: 50, 67417: 50, 31154: 50, 53250: 49, 74789: 49, 91280: 49, 70405: 48, 83450: 48, 39555: 48, 44081: 47, 38601: 47, 29585: 47, 29912: 47

In [23]:
# Making the attribute binary. If the value is less than 2, then set the value to 0. If the value is more than 2 set the value to 1. https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/
SBA_df.loc[:, 'FranchiseCode'] = SBA_df.loc[:, 'FranchiseCode'].where(SBA_df.loc[:, 'FranchiseCode'] > 2, 0)
SBA_df.loc[:, 'FranchiseCode'] = SBA_df.loc[:, 'FranchiseCode'].where(SBA_df.loc[:, 'FranchiseCode'] < 2, 1)

counts = Counter(SBA_df['FranchiseCode'])
print(counts)

Counter({0: 402204, 1: 21295})


### UrbanRural

In [24]:
# Starting with counting the values
counts = Counter(SBA_df['UrbanRural'])
print(counts)

Counter({1: 336764, 2: 81513, 0: 5222})


In [25]:
# Dropping instances with 0, since 0 is undefined. Using function from NewExist
SBA_df = SBA_df.loc[SBA_df['UrbanRural'] != 0]

# Checking the result
counts = Counter(SBA_df['UrbanRural'])
print(counts)

Counter({1: 336764, 2: 81513})


### LowDoc

In [26]:
#Starting with counting the values
counts = Counter(SBA_df['LowDoc'])
print(counts)

Counter({'N': 399618, 'Y': 17716, 'S': 480, 'C': 313, 'A': 103, 'R': 45, '1': 1, '0': 1})


In [27]:
#Dropping rows with values other than 'N' or 'Y'
SBA_df = SBA_df[SBA_df.LowDoc != 'S']
SBA_df = SBA_df[SBA_df.LowDoc != 'C']
SBA_df = SBA_df[SBA_df.LowDoc != 'A']
SBA_df = SBA_df[SBA_df.LowDoc != 'R']
SBA_df = SBA_df[SBA_df.LowDoc != '1']
SBA_df = SBA_df[SBA_df.LowDoc != '0']
counts = Counter(SBA_df['LowDoc'])
print(counts)

Counter({'N': 399618, 'Y': 17716})


In [28]:
# If value equals y then set the value to 1. If value equals N set the value to 0
# Using tho loc function to locate the column, and the where function to replace values. For some reason reversed logic is applicable
SBA_df.loc[:, 'LowDoc'] = SBA_df.loc[:, 'LowDoc'].where(SBA_df.loc[:, 'LowDoc'] != 'Y', 1)
SBA_df.loc[:, 'LowDoc'] = SBA_df.loc[:, 'LowDoc'].where(SBA_df.loc[:, 'LowDoc'] != 'N', 0)

# Cheking if all looks okay
counts = Counter(SBA_df['LowDoc'])
print(counts)

Counter({0: 399618, 1: 17716})


In [29]:
#Changing the Dtype into integer from object. Using the code of line from previously. https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas
SBA_df['LowDoc'] = SBA_df['LowDoc'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417334 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   State              417334 non-null  object
 1   BankState          417334 non-null  object
 2   NAICS              417334 non-null  int64 
 3   ApprovalFY         417334 non-null  int64 
 4   Term               417334 non-null  int64 
 5   NoEmp              417334 non-null  int64 
 6   NewExist           417334 non-null  int64 
 7   FranchiseCode      417334 non-null  int64 
 8   UrbanRural         417334 non-null  int64 
 9   LowDoc             417334 non-null  int64 
 10  DisbursementGross  417334 non-null  int64 
 11  MIS_Status         417334 non-null  object
dtypes: int64(9), object(3)
memory usage: 41.4+ MB


### MIS_Status

In [30]:
# Counting to investigate the data.
counts = Counter(SBA_df['MIS_Status'])
print(counts)

Counter({'P I F': 300489, 'CHGOFF': 116845})


In [31]:
# Using same logic as with the Y and N in LowDoc. 
SBA_df.loc[:, 'MIS_Status'] = SBA_df.loc[:, 'MIS_Status'].where(SBA_df.loc[:, 'MIS_Status'] != 'P I F', 0)
SBA_df.loc[:, 'MIS_Status'] = SBA_df.loc[:, 'MIS_Status'].where(SBA_df.loc[:, 'MIS_Status'] != 'CHGOFF', 1)
counts = Counter(SBA_df['MIS_Status'])
print(counts)

Counter({0: 300489, 1: 116845})


In [32]:
#Changing the Dtype into integer from object. Using the code of line from previously
SBA_df['MIS_Status'] = SBA_df['MIS_Status'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417334 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   State              417334 non-null  object
 1   BankState          417334 non-null  object
 2   NAICS              417334 non-null  int64 
 3   ApprovalFY         417334 non-null  int64 
 4   Term               417334 non-null  int64 
 5   NoEmp              417334 non-null  int64 
 6   NewExist           417334 non-null  int64 
 7   FranchiseCode      417334 non-null  int64 
 8   UrbanRural         417334 non-null  int64 
 9   LowDoc             417334 non-null  int64 
 10  DisbursementGross  417334 non-null  int64 
 11  MIS_Status         417334 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 41.4+ MB


### Term

In [33]:
# Term is already an integer and nothing needs to be done.

### State

In [34]:
#Counting the instances of the different types of values in the column
counts = Counter(SBA_df['State'])
print(counts)

Counter({'CA': 59042, 'TX': 29943, 'NY': 29255, 'FL': 24005, 'PA': 18441, 'OH': 16452, 'IL': 15587, 'MA': 12349, 'NJ': 12168, 'MI': 11859, 'MN': 10912, 'UT': 10882, 'GA': 10584, 'WA': 10481, 'CO': 9614, 'WI': 9217, 'AZ': 9146, 'MO': 7933, 'IN': 7590, 'NC': 7028, 'VA': 6500, 'MD': 6452, 'CT': 5564, 'OR': 5346, 'NH': 5252, 'ID': 4367, 'TN': 4200, 'OK': 3919, 'NV': 3849, 'KS': 3846, 'RI': 3787, 'LA': 3738, 'IA': 3698, 'KY': 3563, 'MS': 2961, 'SC': 2687, 'AL': 2620, 'NE': 2567, 'MT': 2458, 'ME': 2423, 'AR': 2127, 'NM': 2092, 'HI': 1771, 'VT': 1698, 'ND': 1672, 'SD': 1242, 'WV': 1139, 'DE': 1129, 'DC': 817, 'WY': 797, 'AK': 565})


In [35]:
# Assigning each state initial into an integer. Using the replace() function. The same as label encoding.
SBA_df['State'] = SBA_df['State'].str.replace('CA','1')
SBA_df['State'] = SBA_df['State'].str.replace('TX','2')
SBA_df['State'] = SBA_df['State'].str.replace('NY','3')
SBA_df['State'] = SBA_df['State'].str.replace('FL','4')
SBA_df['State'] = SBA_df['State'].str.replace('PA','5')
SBA_df['State'] = SBA_df['State'].str.replace('OH','6')
SBA_df['State'] = SBA_df['State'].str.replace('IL','7')
SBA_df['State'] = SBA_df['State'].str.replace('MA','8')
SBA_df['State'] = SBA_df['State'].str.replace('NJ','9')
SBA_df['State'] = SBA_df['State'].str.replace('MI','10')
SBA_df['State'] = SBA_df['State'].str.replace('MN','11')
SBA_df['State'] = SBA_df['State'].str.replace('UT','12')
SBA_df['State'] = SBA_df['State'].str.replace('GA','13')
SBA_df['State'] = SBA_df['State'].str.replace('WA','14')
SBA_df['State'] = SBA_df['State'].str.replace('CO','15')
SBA_df['State'] = SBA_df['State'].str.replace('WI','16')
SBA_df['State'] = SBA_df['State'].str.replace('AZ','17')

SBA_df['State'] = SBA_df['State'].str.replace('MO','18')
SBA_df['State'] = SBA_df['State'].str.replace('IN','19')
SBA_df['State'] = SBA_df['State'].str.replace('NC','20')
SBA_df['State'] = SBA_df['State'].str.replace('VA','21')
SBA_df['State'] = SBA_df['State'].str.replace('MD','22')
SBA_df['State'] = SBA_df['State'].str.replace('CT','23')
SBA_df['State'] = SBA_df['State'].str.replace('OR','24')
SBA_df['State'] = SBA_df['State'].str.replace('NH','25')
SBA_df['State'] = SBA_df['State'].str.replace('ID','26')
SBA_df['State'] = SBA_df['State'].str.replace('TN','27')
SBA_df['State'] = SBA_df['State'].str.replace('OK','28')
SBA_df['State'] = SBA_df['State'].str.replace('NV','29')
SBA_df['State'] = SBA_df['State'].str.replace('KS','30')
SBA_df['State'] = SBA_df['State'].str.replace('RI','31')
SBA_df['State'] = SBA_df['State'].str.replace('LA','32')
SBA_df['State'] = SBA_df['State'].str.replace('IA','33')
SBA_df['State'] = SBA_df['State'].str.replace('KY','34')
SBA_df['State'] = SBA_df['State'].str.replace('MS','35')
SBA_df['State'] = SBA_df['State'].str.replace('SC','36')
SBA_df['State'] = SBA_df['State'].str.replace('AL','37')
SBA_df['State'] = SBA_df['State'].str.replace('NE','38')
SBA_df['State'] = SBA_df['State'].str.replace('MT','39')
SBA_df['State'] = SBA_df['State'].str.replace('ME','40')

SBA_df['State'] = SBA_df['State'].str.replace('AR','41')
SBA_df['State'] = SBA_df['State'].str.replace('NM','42')
SBA_df['State'] = SBA_df['State'].str.replace('HI','43')
SBA_df['State'] = SBA_df['State'].str.replace('VT','44')
SBA_df['State'] = SBA_df['State'].str.replace('ND','45')
SBA_df['State'] = SBA_df['State'].str.replace('SD','46')
SBA_df['State'] = SBA_df['State'].str.replace('WV','47')

SBA_df['State'] = SBA_df['State'].str.replace('DE','48')
SBA_df['State'] = SBA_df['State'].str.replace('DC','49')
SBA_df['State'] = SBA_df['State'].str.replace('WY','50')
SBA_df['State'] = SBA_df['State'].str.replace('AK','51')
    
counts = Counter(SBA_df['State'])
print(counts)

Counter({'1': 59042, '2': 29943, '3': 29255, '4': 24005, '5': 18441, '6': 16452, '7': 15587, '8': 12349, '9': 12168, '10': 11859, '11': 10912, '12': 10882, '13': 10584, '14': 10481, '15': 9614, '16': 9217, '17': 9146, '18': 7933, '19': 7590, '20': 7028, '21': 6500, '22': 6452, '23': 5564, '24': 5346, '25': 5252, '26': 4367, '27': 4200, '28': 3919, '29': 3849, '30': 3846, '31': 3787, '32': 3738, '33': 3698, '34': 3563, '35': 2961, '36': 2687, '37': 2620, '38': 2567, '39': 2458, '40': 2423, '41': 2127, '42': 2092, '43': 1771, '44': 1698, '45': 1672, '46': 1242, '47': 1139, '48': 1129, '49': 817, '50': 797, '51': 565})


In [36]:
# Converting the attribute into an integer
SBA_df['State'] = SBA_df['State'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417334 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   State              417334 non-null  int64 
 1   BankState          417334 non-null  object
 2   NAICS              417334 non-null  int64 
 3   ApprovalFY         417334 non-null  int64 
 4   Term               417334 non-null  int64 
 5   NoEmp              417334 non-null  int64 
 6   NewExist           417334 non-null  int64 
 7   FranchiseCode      417334 non-null  int64 
 8   UrbanRural         417334 non-null  int64 
 9   LowDoc             417334 non-null  int64 
 10  DisbursementGross  417334 non-null  int64 
 11  MIS_Status         417334 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 41.4+ MB


### BankState

In [37]:
# Getting an overview of the Attribute.
counts = Counter(SBA_df['BankState'])
print(counts)

Counter({'CA': 56545, 'NC': 55923, 'IL': 36450, 'OH': 31413, 'RI': 29066, 'VA': 24344, 'SD': 19390, 'NY': 18801, 'DE': 16930, 'TX': 11940, 'UT': 10881, 'MN': 7174, 'PA': 6458, 'WI': 6330, 'FL': 5617, 'MO': 5145, 'MA': 5017, 'OR': 4403, 'GA': 4184, 'AL': 3958, 'NJ': 3571, 'WA': 3374, 'CT': 3350, 'IN': 3206, 'CO': 3151, 'IA': 2843, 'MI': 2533, 'OK': 2489, 'KS': 2347, 'SC': 2308, 'AR': 2176, 'NH': 2031, 'NE': 2001, 'MT': 1905, 'MS': 1895, 'ID': 1845, 'MD': 1792, 'ND': 1617, 'TN': 1534, 'NV': 1412, 'KY': 1396, 'AZ': 1322, 'HI': 1310, 'ME': 1298, 'NM': 1164, 'LA': 1152, 'VT': 1141, 'WY': 468, 'WV': 342, 'AK': 197, 'DC': 192, 'GU': 2, 'PR': 1})


In [38]:
# There are Bank states with just one instance, to avoid overfitting these instances are removed. Removing: 'GU' and 'PR'
# Using the filter function isin() https://sparkbyexamples.com/pandas/pandas-isin-explained-with-examples/
SBA_df.drop(SBA_df[SBA_df['BankState'].isin(['PR', 'GU'])].index, inplace=True)

counts = Counter(SBA_df['BankState'])
print(counts)


Counter({'CA': 56545, 'NC': 55923, 'IL': 36450, 'OH': 31413, 'RI': 29066, 'VA': 24344, 'SD': 19390, 'NY': 18801, 'DE': 16930, 'TX': 11940, 'UT': 10881, 'MN': 7174, 'PA': 6458, 'WI': 6330, 'FL': 5617, 'MO': 5145, 'MA': 5017, 'OR': 4403, 'GA': 4184, 'AL': 3958, 'NJ': 3571, 'WA': 3374, 'CT': 3350, 'IN': 3206, 'CO': 3151, 'IA': 2843, 'MI': 2533, 'OK': 2489, 'KS': 2347, 'SC': 2308, 'AR': 2176, 'NH': 2031, 'NE': 2001, 'MT': 1905, 'MS': 1895, 'ID': 1845, 'MD': 1792, 'ND': 1617, 'TN': 1534, 'NV': 1412, 'KY': 1396, 'AZ': 1322, 'HI': 1310, 'ME': 1298, 'NM': 1164, 'LA': 1152, 'VT': 1141, 'WY': 468, 'WV': 342, 'AK': 197, 'DC': 192})


In [39]:
# Same approach as in 'State' but new sorting
SBA_df['BankState'] = SBA_df['BankState'].str.replace('CA','1')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NC','2')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('IL','3')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('OH','4')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('RI','5')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('VA','6')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('SD','7')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NY','8')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('DE','9')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('TX','10')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('UT','11')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MN','12')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('PA','13')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('WI','14')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('FL','15')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MO','16')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MA','17')

SBA_df['BankState'] = SBA_df['BankState'].str.replace('OR','18')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('GA','19')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('AL','20')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NJ','21')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('WA','22')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('CT','23')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('IN','24')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('CO','25')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('IA','26')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MI','27')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('OK','28')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('KS','29')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('SC','30')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('AR','31')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NH','32')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NE','33')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MT','34')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MS','35')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('ID','36')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('MD','37')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('ND','38')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('TN','39')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NV','40')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('KY','41')

SBA_df['BankState'] = SBA_df['BankState'].str.replace('AZ','42')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('HI','43')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('ME','44')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('NM','45')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('LA','46')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('VT','47')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('WY','48')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('WV','49')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('AK','50')
SBA_df['BankState'] = SBA_df['BankState'].str.replace('DC','51')

    
counts = Counter(SBA_df['BankState'])
print(counts)

Counter({'1': 56545, '2': 55923, '3': 36450, '4': 31413, '5': 29066, '6': 24344, '7': 19390, '8': 18801, '9': 16930, '10': 11940, '11': 10881, '12': 7174, '13': 6458, '14': 6330, '15': 5617, '16': 5145, '17': 5017, '18': 4403, '19': 4184, '20': 3958, '21': 3571, '22': 3374, '23': 3350, '24': 3206, '25': 3151, '26': 2843, '27': 2533, '28': 2489, '29': 2347, '30': 2308, '31': 2176, '32': 2031, '33': 2001, '34': 1905, '35': 1895, '36': 1845, '37': 1792, '38': 1617, '39': 1534, '40': 1412, '41': 1396, '42': 1322, '43': 1310, '44': 1298, '45': 1164, '46': 1152, '47': 1141, '48': 468, '49': 342, '50': 197, '51': 192})


In [40]:
# COnverting into an integer and checking if it worked.
SBA_df['BankState'] = SBA_df['BankState'].astype(np.int64)
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417331 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   State              417331 non-null  int64
 1   BankState          417331 non-null  int64
 2   NAICS              417331 non-null  int64
 3   ApprovalFY         417331 non-null  int64
 4   Term               417331 non-null  int64
 5   NoEmp              417331 non-null  int64
 6   NewExist           417331 non-null  int64
 7   FranchiseCode      417331 non-null  int64
 8   UrbanRural         417331 non-null  int64
 9   LowDoc             417331 non-null  int64
 10  DisbursementGross  417331 non-null  int64
 11  MIS_Status         417331 non-null  int64
dtypes: int64(12)
memory usage: 41.4 MB


### NAICS

In [41]:
# Starting with looking on the NAICS to get an overview.
counts = Counter(SBA_df['NAICS'])
print(counts)

Counter({722110: 18878, 722211: 15506, 812112: 6938, 811111: 6424, 561730: 6145, 621210: 5263, 238990: 5009, 447110: 4912, 621111: 4737, 484110: 4622, 453998: 4581, 236118: 4215, 621310: 4110, 238220: 3977, 721110: 3929, 624410: 3900, 541110: 3701, 445110: 3664, 713940: 3636, 236115: 3503, 812320: 3492, 238210: 3483, 445310: 3397, 531210: 3383, 812990: 3271, 722213: 3106, 524210: 2967, 484121: 2955, 541611: 2917, 453220: 2774, 561720: 2748, 445120: 2708, 541330: 2548, 541512: 2423, 811121: 2417, 541511: 2405, 451110: 2360, 812199: 2271, 445299: 2173, 442110: 2112, 541990: 2107, 541613: 2084, 441310: 1978, 541211: 1973, 541519: 1906, 238910: 1895, 541940: 1889, 722410: 1885, 541618: 1824, 811192: 1803, 423990: 1756, 448190: 1744, 448310: 1693, 448120: 1684, 441120: 1678, 812310: 1643, 238320: 1634, 453110: 1633, 522310: 1620, 452990: 1552, 332710: 1519, 621340: 1507, 236220: 1487, 561990: 1481, 713990: 1473, 484220: 1472, 446120: 1448, 424990: 1438, 722320: 1435, 541810: 1406, 541430: 1

In [42]:
# All the NAICS codes are 6 digits. The first two digits determines the broad industry sector. https://davenport.libguides.com/naics
# Removing the last 4 digits by dividing with 10000. Dropping instances with values less than 10. https://www.datasciencelearner.com/divide-two-columns-in-pandas-methods/

SBA_df['NAICS']/=10000

SBA_df['NAICS'] = SBA_df['NAICS'][SBA_df['NAICS'] >= 10]

counts = Counter(SBA_df['NAICS'])
print(counts)

Counter({72.211: 18878, 72.2211: 15506, 81.2112: 6938, 81.1111: 6424, 56.173: 6145, 62.121: 5263, 23.899: 5009, 44.711: 4912, 62.1111: 4737, 48.411: 4622, 45.3998: 4581, 23.6118: 4215, 62.131: 4110, 23.822: 3977, 72.111: 3929, 62.441: 3900, 54.111: 3701, 44.511: 3664, 71.394: 3636, 23.6115: 3503, 81.232: 3492, 23.821: 3483, 44.531: 3397, 53.121: 3383, 81.299: 3271, 72.2213: 3106, 52.421: 2967, 48.4121: 2955, 54.1611: 2917, 45.322: 2774, 56.172: 2748, 44.512: 2708, 54.133: 2548, 54.1512: 2423, 81.1121: 2417, 54.1511: 2405, 45.111: 2360, 81.2199: 2271, 44.5299: 2173, 44.211: 2112, 54.199: 2107, 54.1613: 2084, 44.131: 1978, 54.1211: 1973, 54.1519: 1906, 23.891: 1895, 54.194: 1889, 72.241: 1885, 54.1618: 1824, 81.1192: 1803, 42.399: 1756, 44.819: 1744, 44.831: 1693, 44.812: 1684, 44.112: 1678, 81.231: 1643, 23.832: 1634, 45.311: 1633, 52.231: 1620, 45.299: 1552, 33.271: 1519, 62.134: 1507, 23.622: 1487, 56.199: 1481, 71.399: 1473, 48.422: 1472, 44.612: 1448, 42.499: 1438, 72.232: 1435, 54.

In [43]:
# Removing the decimals. Keeping in mind, that we are dealing with categories and not traditional numbers.
# The decimals should be rounded down. https://datatofish.com/round-values-pandas-dataframe/
SBA_df['NAICS'] = SBA_df['NAICS'].apply(np.floor)

counts = Counter(SBA_df['NAICS'])
print(counts)

Counter({44.0: 50908, 72.0: 46484, 54.0: 43472, 23.0: 41840, 81.0: 40401, 62.0: 28385, 42.0: 25907, 56.0: 23213, 45.0: 22510, 33.0: 19004, 48.0: 15285, 53.0: 9843, 32.0: 9427, 71.0: 8504, 51.0: 7463, 52.0: 7396, 31.0: 6615, 61.0: 4938, 11.0: 2815, 49.0: 1597, 21.0: 811, 22.0: 337, 55.0: 94, 92.0: 76, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})


In [44]:
# Dropping instances with the 'value' of nan.
SBA_df.dropna(inplace=True)

counts = Counter(SBA_df['NAICS'])
print(counts)

Counter({44.0: 50908, 72.0: 46484, 54.0: 43472, 23.0: 41840, 81.0: 40401, 62.0: 28385, 42.0: 25907, 56.0: 23213, 45.0: 22510, 33.0: 19004, 48.0: 15285, 53.0: 9843, 32.0: 9427, 71.0: 8504, 51.0: 7463, 52.0: 7396, 31.0: 6615, 61.0: 4938, 11.0: 2815, 49.0: 1597, 21.0: 811, 22.0: 337, 55.0: 94, 92.0: 76})


In [45]:
# Converting the attribute into integers
SBA_df['NAICS'] = SBA_df['NAICS'].astype(np.int64)

SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417325 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   State              417325 non-null  int64
 1   BankState          417325 non-null  int64
 2   NAICS              417325 non-null  int64
 3   ApprovalFY         417325 non-null  int64
 4   Term               417325 non-null  int64
 5   NoEmp              417325 non-null  int64
 6   NewExist           417325 non-null  int64
 7   FranchiseCode      417325 non-null  int64
 8   UrbanRural         417325 non-null  int64
 9   LowDoc             417325 non-null  int64
 10  DisbursementGross  417325 non-null  int64
 11  MIS_Status         417325 non-null  int64
dtypes: int64(12)
memory usage: 41.4 MB


In [46]:
SBA_df.head()

State  BankState  NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12     20         12     44        2006   162      2         2              1   
24     41         35     72        2006   126      7         1              0   
28      1          7     61        2006    83     18         2              0   
30      4         15     23        2006    84      4         1              0   
32     32         46     53        2006    60      3         1              0   

    UrbanRural  LowDoc  DisbursementGross  MIS_Status  
12           1       0             253400           0  
24           1       0             137300           0  
28           1       0             438541           0  
30           1       0              51440           0  
32           1       0              50000           0

### NoEmp

In [47]:
# NoEmp looks fine. The values looks reasonable and seems realistic. No preprocessing required.
counts = Counter(SBA_df['NoEmp'])
print(counts)

Counter({1: 91408, 2: 75151, 3: 45814, 4: 35258, 5: 28284, 6: 20134, 7: 13588, 8: 13165, 10: 12852, 12: 8057, 9: 7094, 15: 6974, 0: 5207, 20: 5156, 11: 4367, 14: 3684, 25: 3411, 13: 3287, 30: 2704, 18: 2599, 16: 2512, 17: 1962, 22: 1611, 40: 1548, 35: 1481, 50: 1190, 21: 1150, 19: 1138, 24: 1086, 23: 1033, 45: 841, 28: 835, 26: 810, 27: 672, 60: 657, 32: 651, 29: 390, 65: 351, 55: 351, 36: 348, 42: 347, 80: 345, 38: 336, 34: 335, 33: 333, 100: 330, 70: 321, 31: 319, 75: 303, 48: 237, 37: 233, 43: 224, 39: 174, 47: 172, 41: 169, 44: 167, 46: 157, 52: 153, 150: 150, 90: 148, 85: 141, 49: 129, 120: 114, 110: 103, 51: 99, 53: 93, 54: 91, 56: 89, 62: 87, 200: 86, 125: 81, 130: 80, 95: 78, 58: 77, 68: 75, 57: 67, 63: 63, 82: 62, 140: 57, 72: 53, 105: 47, 64: 45, 67: 44, 59: 43, 61: 41, 71: 41, 300: 40, 73: 40, 78: 39, 74: 38, 66: 37, 250: 36, 88: 36, 98: 35, 160: 34, 77: 34, 115: 33, 76: 29, 86: 28, 180: 28, 79: 26, 170: 25, 135: 25, 84: 25, 69: 24, 87: 24, 145: 21, 102: 20, 97: 20, 92: 19, 

### The preprocessed dataset

In [48]:
# Displyaing how the dataset looks like
SBA_df.head()

State  BankState  NAICS  ApprovalFY  Term  NoEmp  NewExist  FranchiseCode  \
12     20         12     44        2006   162      2         2              1   
24     41         35     72        2006   126      7         1              0   
28      1          7     61        2006    83     18         2              0   
30      4         15     23        2006    84      4         1              0   
32     32         46     53        2006    60      3         1              0   

    UrbanRural  LowDoc  DisbursementGross  MIS_Status  
12           1       0             253400           0  
24           1       0             137300           0  
28           1       0             438541           0  
30           1       0              51440           0  
32           1       0              50000           0

In [49]:
# Displyaing the Dtypes and the amount of instances in the dataset
SBA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417325 entries, 12 to 894831
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   State              417325 non-null  int64
 1   BankState          417325 non-null  int64
 2   NAICS              417325 non-null  int64
 3   ApprovalFY         417325 non-null  int64
 4   Term               417325 non-null  int64
 5   NoEmp              417325 non-null  int64
 6   NewExist           417325 non-null  int64
 7   FranchiseCode      417325 non-null  int64
 8   UrbanRural         417325 non-null  int64
 9   LowDoc             417325 non-null  int64
 10  DisbursementGross  417325 non-null  int64
 11  MIS_Status         417325 non-null  int64
dtypes: int64(12)
memory usage: 41.4 MB


In [50]:
#Exporting the preprocessed data to better handle it in a new jupyter Notebook.
SBA_df.to_csv('SBA_PreprocessedData.csv')